In [2]:
import requests
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# API같은 경우에는 이런 keyword 바꾸면 다른거 검색가능합니다..
# API가 아닐 경우에도 동일할 가능성이 커요
keyword = '세탁기'
keys = []

with open('API_KEY.txt', 'r') as key:
    for line in key.readlines():
        text = line.strip()
        if text != '':
            keys.append(text.split(':')[1])

headers = {
    'X-Naver-Client-Id' : keys[0],
    'X-Naver-Client-Secret' : keys[1],
}
            
url = f'https://openapi.naver.com/v1/search/shop?&query={keyword}'
time.sleep(5.5)
text = requests.get(url, headers=headers)
line = text.json()

In [9]:
columns = ['title', 'link', 'image', 'price', 'maker', 'category1', 'category2']

item_list = []

for item in line['items']:
    time.sleep(7.7)
#     item_list.append([item['title'], item['link'], item['image'], item['lprice'], item['maker'], item['category3'], item['category4']])    
    data = requests.get(item['link'])
    soup = BeautifulSoup(data.text)
    
    text = soup.select_one('#wrap > div.product_bridge_product__n_89z > a:nth-child(5)')
    
    time.sleep(5.5)
    review = requests.get(text.attrs['href'])
    
    soup = BeautifulSoup(review.text)
    reviews = soup.select('#section_review > ul li > div.reviewItems_review__DqLYb > div.reviewItems_review_text__dq0kE > p')
    break
    

In [26]:
reviews[2].get_text()

'이전 사용 세탁기는 뒤판에 접지선이 있었는데  새 세탁기는 접지선이 없어요. 접지가 된건지 기사님이 설명을 따로 해 주지 않으셨어요. 세탁기 밑에 스티로폼이 끼어 있길래 물어보니까 그건 빼지 말고 사용하라고 하시더라구요. 세탁기 설치하실때 그런건 좀 설명해주시면 더 좋을것 같아요.고객이 묻지 않으면 얘기해 주시지 않더라구요.  세탁기 사용방법에 대한 설명은 해주셨어요. 세탁기 사용해보니  소음은 보통이에요. 이불빨래는  두께가 있는건 1개, 얇은 여름이불은 2~3개정도 들어갈것 같네요. 이전에 드럼세탁기 10kg 사용했는데, 요 새 세탁기는 그것보다 크기가 슬림해서 좋아요. 공간이 남아요. 용량은 4인 가족도 사용할수 있겠어요. 대신 자주 세탁기를 돌려야 하겠죠~헹굼 횟수가 3회뿐이라서  헹굼  추가하기 원하면 헹굼 3회 끝난후 다시 헹굼 1회 버튼을 눌러야한다는 불편함이 있네요. 전체적인 디자인과 기능은 괜찮네요~'

In [22]:
import re
def tokenize_and_stemming(sentence, tokenizer):
    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence) 
    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming
    sentence_tokenized = []
    for token, pos in raw_pos_tagged:
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)  
    return sentence_tokenized
